Scrapping all the reports (Cause of Collision) at https://www.ntsb.gov/investigations/AccidentReports/Pages/AccidentReports.aspx

In [13]:
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from bs4 import BeautifulSoup
import urllib.request
import requests

driver = webdriver.Chrome('/Users/m804/Desktop/chromedriver')
url = 'https://www.ntsb.gov/investigations/AccidentReports/Pages/aviation.aspx'
base = 'https://www.ntsb.gov/investigations/AccidentReports/Pages/'
driver.get(url)
driver1 = webdriver.Chrome('/Users/m804/Desktop/chromedriver')

S = []
counter = 1
while True:
    try:
        link = driver.find_element_by_id("ntsb-tbl_next")
    except NoSuchElementException:
        break
    link.click()
    html = driver.find_element_by_tag_name('html').get_attribute('innerHTML')
    soup = BeautifulSoup(html, "lxml")
    List = soup.find_all('td', class_ = 'ms-vb reporttitle ')
    print("scraping page " + str(counter))
    counter += 1
    if counter == 50:
        print("finished")
        break;
    for item in List:
        try:
            newurl = base + item.contents[0]['href']
            #print(newurl)
            driver1.get(newurl)
            html1 = driver1.find_element_by_tag_name('html').get_attribute('innerHTML')
            #print("New HTML!!!")
            soup = BeautifulSoup(html1, "lxml")
            summary = soup.find_all('div', class_ = 'ms-rtestate-field')[-1].contents[0]
            summary = (list(map(lambda x:x.strip(),summary)))[0]
            if summary != '\u200b':
                summary = summary.replace('\u200b','')
                S.append(summary)
        except TypeError:
            pass
        except IndexError:
            pass

scraping page 1
scraping page 2
scraping page 3
scraping page 4
scraping page 5
scraping page 6
scraping page 7
scraping page 8
scraping page 9
scraping page 10
scraping page 11
scraping page 12
scraping page 13
scraping page 14
scraping page 15
scraping page 16
scraping page 17
scraping page 18
scraping page 19
scraping page 20
scraping page 21
scraping page 22
scraping page 23
scraping page 24
scraping page 25
scraping page 26
scraping page 27
scraping page 28
scraping page 29
scraping page 30
scraping page 31
scraping page 32
scraping page 33
scraping page 34
scraping page 35
scraping page 36
scraping page 37
scraping page 38
scraping page 39
scraping page 40
scraping page 41
scraping page 42
scraping page 43
scraping page 44
scraping page 45
scraping page 46
scraping page 47
scraping page 48
scraping page 49
finished


In [14]:
S

['The National Transportation Safety Board determines that the probable cause of this accident was the flight crew’s mismanagement of the approach and multiple deviations from company standard operating procedures, which placed the airplane in an unsafe situation and led to an unstabilized approach, a descent below minimum descent altitude without visual contact with the runway environment, and an aerodynamic stall. Contributing to the accident were Execuflight’s casual attitude toward compliance with standards; its inadequate hiring, training, and operational oversight of the flight crew; the company’s lack of a formal safety program; and the Federal Aviation Administration’s insufficient oversight of the company’s training program and flight operations.',
 'The National Transportation Safety Board determines that the probable cause of this accident was the captain’s inability to maintain directional control of the airplane due to his application of excessive reverse thrust, which deg

In [17]:
# Preprocessing of the data by stemming 
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
import nltk
nltk.download('punkt')

ps = PorterStemmer()
new_report = []
word_data = []

insignificant_terms = ['us','were','our','we','NMAC','air','data','follow','in','the','had','for','from','on','to','with','and','while','','than','less','in','of','at','an','has']
for i in range(0,len(S)):
    temp_report = ''
    # Stem the narratives of each report
    if pd.isnull(S[i]) != True:
        words = word_tokenize(S[i])
        for w in words:
            if w not in word_data:
                word_data.append([w, ps.stem(w)])
            w = ps.stem(w)
            # Add the insignificant words here 
            if w not in insignificant_terms:
                temp_report += (w + " ")
        new_report.append((temp_report))

[nltk_data] Downloading package punkt to /Users/m804/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [18]:
new_report

['nation transport safeti board determin that probabl caus thi accid wa flight crew ’ s mismanag approach multipl deviat compani standard oper procedur , which place airplan unsaf situat led unstabil approach , a descent below minimum descent altitud without visual contact runway environ , aerodynam stall . contribut accid execuflight ’ s casual attitud toward complianc standard ; it inadequ hire , train , oper oversight flight crew ; compani ’ s lack a formal safeti program ; feder aviat administr ’ s insuffici oversight compani ’ s train program flight oper . ',
 'nation transport safeti board determin that probabl caus thi accid wa captain ’ s inabl maintain direct control airplan due hi applic excess revers thrust , which degrad effect rudder control airplan ’ s head . contribut accid captain ’ s ( 1 ) situat stress result hi concern about stop perform ( 2 ) attent limit due high workload dure land , which prevent him immedi recogn use excess revers thrust . ',
 'ntsb determin that

In [19]:
import pandas as pd
result = pd.DataFrame({'Summary':S})
result = result.reset_index()
result.to_csv('Reports_stemmed.csv', index=False)